This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'X_XXXXXXXXX'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key='+API_KEY+'&start_date=1970-01-01&end_date=1970-01-01')

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2018-09-03T22:28:07.555Z',
  'newest_available_date': '2018-09-03',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2000-06-07',
  'end_date': '1970-01-01',
  'data': [],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# Collect data from the Franfurt Stock Exchange
afx = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key='+API_KEY+'&start_date=2017-01-01&end_date=2017-12-31')

# Convert to dictionary
afx_dict = dict(afx.json())

In [6]:
# turn data lists into dictionary
columns = afx_dict['dataset']['column_names']
data = afx_dict['dataset']['data']

afx_data = {}
for i, day in enumerate(data):
    afx_data[i] = dict(zip(columns,day))

In [7]:
# Get highest opening price
high_open = max([afx_data[i]['Open'] for i in afx_data if afx_data[i]['Open'] is not None])

# Get lowest opening price
low_open = min([afx_data[i]['Open'] for i in afx_data if afx_data[i]['Open'] is not None])

print('Highest opening price:', high_open)
print('Lowest opening price:', low_open)

Highest opening price: 53.11
Lowest opening price: 34.0


In [8]:
# Calculate the largest change in one day
change = [afx_data[i]['Close'] - afx_data[i]['Open'] 
                                   for i in afx_data if afx_data[i]['Open'] is not None 
                                   and [afx_data[i]['Close']] is not None]

print('Largest positive change in one day: {:.4f}'.format(max(change)))
print('Largest negative change in one day: {:.4f}'.format(min(change)))

Largest positive change in one day: 1.6400
Largest negative change in one day: -1.6900


In [9]:
# Calculate the largest change between any two days
two_day_change = []

for i in afx_data:
    if i+1 < len(afx_data):
        if afx_data[i]['Close'] is not None:
            two_day_change.extend([afx_data[i+1]['Close'] - afx_data[i]['Close']])
        else:
            two_day_change.extend(0)

print('Largest postive change between two days: {:.4f}'.format(max(two_day_change)))
print('Largest negative change between two days: {:.4f}'.format(min(two_day_change)))

Largest postive change between two days: 2.5600
Largest negative change between two days: -1.7200


In [10]:
# Calculate the average daily traded volume
traded_vol = [afx_data[i]['Traded Volume'] for i in afx_data if afx_data[i]['Traded Volume'] is not None]

print('Mean traded volume: {:.4f}'.format(sum(traded_vol)/len(traded_vol)))

Mean traded volume: 89124.3373


In [11]:
# Calculate the median trading volume
median_trade_vol = sorted(traded_vol)[len(traded_vol)//2]
print('Median traded volume: {:.2f}'.format(median_trade_vol))

Median traded volume: 76286.00


In [12]:
# Calculate Daily Traded Volume
daily_traded_vol = []
for i in afx_data:
    if i+1 < len(afx_data):
        if afx_data[i]['Traded Volume'] is not None:
            daily_traded_vol.extend([afx_data[i+1]['Traded Volume'] - afx_data[i]['Traded Volume']])
        else:
            daily_traded_vol.extend(0)

# Calculate the mean daily trading volumne
print('Mean daily trading volume: {:.4f}'.format(sum(daily_traded_vol)/len(daily_traded_vol)))

Mean daily trading volume: 39.6063


In [13]:
# Calculate the median daily trading volume
median_daily_trade_vol = sorted(daily_traded_vol)[len(daily_traded_vol)//2]
print('Median daily trading volume: {:.4f}'.format(median_daily_trade_vol))

Median daily trading volume: 65.0000
